In [1]:
import pandas as pd
date_str = '2021-12-06'

In [20]:
begin = 1990
end = 2000

In [3]:
%%time
df_Ref_New = pd.read_csv('./Data-Metrics/MID_MAG_Journal_References_Year.csv', sep='\t')
df_Ref_New

CPU times: user 4min 56s, sys: 56.1 s, total: 5min 52s
Wall time: 5min 55s


,Citing,Cited,Citing_Year,Cited_Year
0,108,2155007194,2013.0,2008.0
1,147,132242577,2008.0,2001.0
2,147,1973725016,2008.0,2007.0
3,147,2084894991,2008.0,2007.0
4,147,2091764496,2008.0,2007.0
...,...,...,...,...
1192476649,3182980714,2989526080,2021.0,2020.0
1192476650,3182980714,2996077625,2021.0,2020.0
1192476651,3217810308,2090392569,2021.0,2001.0
1192476652,3217810308,2318721171,2021.0,1991.0


In [4]:
%%time
df_Ref_New1 = df_Ref_New.loc[df_Ref_New['Citing_Year'] <= end, :]
df_Ref_New2 = df_Ref_New1.loc[df_Ref_New1['Citing_Year'] >= begin, :]
df_Ref_New2

CPU times: user 12.2 s, sys: 5.45 s, total: 17.6 s
Wall time: 17.8 s


,Citing,Cited,Citing_Year,Cited_Year
87,587,125816956,1998.0,1970.0
88,587,1964317701,1998.0,1987.0
89,587,1968472516,1998.0,1935.0
90,587,1972948751,1998.0,1971.0
91,587,1988218481,1998.0,1982.0
...,...,...,...,...
1192474171,3217243181,2041408515,1997.0,1998.0
1192474935,2141799708,2105291102,1997.0,1990.0
1192474936,2141799708,2121499252,1997.0,1993.0
1192475202,3217470764,2007497347,1993.0,2012.0


In [10]:
%%time
D = {}
from tqdm import tqdm
df_Ref_New_Y1 = df_Ref_New2.groupby(['Citing_Year'], as_index=False, sort=False)
for name, df_group in tqdm(df_Ref_New_Y1):
    df_Ref_New_Y2 = df_group.groupby(['Cited_Year'], as_index=False, sort=False)
    for name2, df_sec_group in tqdm(df_Ref_New_Y2):
        D[(name, name2)] = list(zip(df_sec_group['Citing'].values,df_sec_group['Cited'].values))
        # print(name, name2, len(D[(name, name2)]))
        # break
    # break

100%|██████████| 11/11 [00:49<00:00,  4.51s/it]

CPU times: user 40.1 s, sys: 12.4 s, total: 52.5 s
Wall time: 53.6 s


In [11]:
D[(1998,1970)][:10]

[(587, 125816956),
 (587, 2110402679),
 (67869, 2318073985),
 (117927, 2020596661),
 (117927, 2091372497),
 (148591, 2028945667),
 (260791, 2070565867),
 (300031, 1982873210),
 (307978, 2024294917),
 (332613, 2100837269)]

In [12]:
%%time
from collections import defaultdict
Ori_Dict = defaultdict(list)

Citing_List = list(df_Ref_New2['Citing'].values)
Cited_List = list(df_Ref_New2['Cited'].values)
List_Len = len(Citing_List)

for i in tqdm(range(List_Len)):
    Ori_Dict[Citing_List[i]].append(Cited_List[i])

100%|██████████| 116227249/116227249 [04:15<00:00, 454527.28it/s] 

CPU times: user 3min 12s, sys: 33.5 s, total: 3min 45s
Wall time: 4min 27s


In [13]:
from collections import defaultdict
Ref_List = [defaultdict(list) for i in range(10)]
Ref_List

[defaultdict(list, {}),
 defaultdict(list, {}),
 defaultdict(list, {}),
 defaultdict(list, {}),
 defaultdict(list, {}),
 defaultdict(list, {}),
 defaultdict(list, {}),
 defaultdict(list, {}),
 defaultdict(list, {}),
 defaultdict(list, {})]

In [14]:
%%time
import random 
from tqdm import tqdm


for k,v in tqdm(D.items()):
    # Control the same year for each citing - cited pair.
    origin = [i[0] for i in v]
    temp = [i[1] for i in v]

    cur_len = len(origin)

    for epoch in range(10):
        random.shuffle(temp)
        # Single shuffle for the whole list is equivalent to Q times E switches in the null model.
        for i in range(cur_len):
            Ref_List[epoch][origin[i]].append(temp[i])

100%|██████████| 2440/2440 [37:05<00:00,  1.10it/s]  

CPU times: user 30min 47s, sys: 1min 56s, total: 32min 43s
Wall time: 37min 5s


In [15]:
Ref_List[9][587]

[2091913250,
 2006650073,
 2091069417,
 2086125534,
 2336320667,
 2166020098,
 2090285010,
 2395825358,
 2001699614,
 2403765287,
 2069306609,
 87082738,
 1985655368]

In [16]:
%%time
df_MAG_Sel_wRC = pd.read_csv("../Final-Data/Data-Tables-MAG_" + date_str + "/df_MAG_Sel_wRC.csv", sep='\t')
df_MAG_Sel_wRC
# Sel_MAG_Ref_Cite.ipynb

CPU times: user 2min 32s, sys: 27.2 s, total: 2min 59s
Wall time: 3min


,MAGID,DOI,DocType,Year,Date,JID,ConfSID,ConfInsID,C_all,C10,R,C3,C5
0,2789336,NaN,Journal,2008.0,2008-01-01,2.048520e+08,NaN,NaN,1.0,1.0,13.0,1.0,1.0
1,9552966,NaN,Journal,2005.0,2005-05-01,2.737569e+09,NaN,NaN,0.0,0.0,37.0,0.0,0.0
2,9694411,NaN,Conference,1994.0,1994-01-01,NaN,2.755927e+09,NaN,0.0,0.0,0.0,0.0,0.0
3,14355006,NaN,Repository,2011.0,2011-01-01,3.121261e+09,NaN,NaN,0.0,0.0,0.0,0.0,0.0
4,14558443,10.1007/978-3-662-45174-8_28,Conference,2014.0,2014-10-12,NaN,1.131603e+09,4.038532e+06,14.0,14.0,11.0,6.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134129183,3215499129,10.1109/ICRERA52334.2021.9598593,NaN,2021.0,2021-09-26,NaN,NaN,NaN,0.0,0.0,14.0,0.0,0.0
134129184,3215845741,10.3389/FCVM.2021.766676,Journal,2021.0,2021-11-24,2.595414e+09,NaN,NaN,0.0,0.0,33.0,0.0,0.0
134129185,3216186450,10.1016/J.IDNOW.2021.11.004,NaN,2021.0,2021-11-24,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
134129186,3216762448,10.1016/J.ISTRUC.2021.10.095,Journal,2022.0,2022-01-01,2.898443e+09,NaN,NaN,0.0,0.0,33.0,0.0,0.0


In [17]:
%%time
# df_MAG_J = df_MAG_Sel_wRC.loc[pd.notnull(df_MAG_Sel_wRC['JID']),:]
df_MAG_J = df_MAG_Sel_wRC.loc[df_MAG_Sel_wRC['DocType'] == 'Journal', :]
df_MAG_J

CPU times: user 13.8 s, sys: 4.06 s, total: 17.9 s
Wall time: 18.1 s


,MAGID,DOI,DocType,Year,Date,JID,ConfSID,ConfInsID,C_all,C10,R,C3,C5
0,2789336,NaN,Journal,2008.0,2008-01-01,2.048520e+08,NaN,NaN,1.0,1.0,13.0,1.0,1.0
1,9552966,NaN,Journal,2005.0,2005-05-01,2.737569e+09,NaN,NaN,0.0,0.0,37.0,0.0,0.0
7,19586282,NaN,Journal,2007.0,2007-07-01,1.319676e+08,NaN,NaN,0.0,0.0,0.0,0.0,0.0
9,23032558,NaN,Journal,1997.0,1997-10-01,2.755314e+09,NaN,NaN,4.0,3.0,0.0,0.0,1.0
14,33698157,NaN,Journal,1961.0,1961-12-31,1.052089e+08,NaN,NaN,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134129174,3211627098,NaN,Journal,2000.0,2000-01-01,1.004905e+09,NaN,NaN,0.0,0.0,0.0,0.0,0.0
134129175,3212006038,NaN,Journal,2019.0,2019-07-22,7.630495e+07,NaN,NaN,0.0,0.0,0.0,0.0,0.0
134129182,3215152208,10.11646/ZOOTAXA.5072.6.3,Journal,2021.0,2021-12-03,1.714719e+08,NaN,NaN,0.0,0.0,0.0,0.0,0.0
134129184,3215845741,10.3389/FCVM.2021.766676,Journal,2021.0,2021-11-24,2.595414e+09,NaN,NaN,0.0,0.0,33.0,0.0,0.0


In [18]:
%%time
len(df_MAG_J["JID"].unique())

CPU times: user 1.04 s, sys: 6.7 ms, total: 1.05 s
Wall time: 1.1 s


48995

In [19]:
%%time
Dict_MagID_2_JID_New = dict(list(zip(df_MAG_J['MAGID'].values, df_MAG_J['JID'].values.astype(int))))
print(len(Dict_MagID_2_JID_New))
Dict_MagID_2_JID_New[9552966]

90764813
CPU times: user 45.9 s, sys: 11 s, total: 56.9 s
Wall time: 1min 2s


2737568815

In [21]:
%%time
df_MAG_J1 = df_MAG_J.loc[df_MAG_J['Year'] <= end, :]
df_MAG_J2 = df_MAG_J1.loc[df_MAG_J1['Year'] >= begin, :]
df_MAG_J2

CPU times: user 6.2 s, sys: 3.66 s, total: 9.86 s
Wall time: 10.2 s


,MAGID,DOI,DocType,Year,Date,JID,ConfSID,ConfInsID,C_all,C10,R,C3,C5
9,23032558,NaN,Journal,1997.0,1997-10-01,2.755314e+09,NaN,NaN,4.0,3.0,0.0,0.0,1.0
15,35428871,10.1023/A:1022270923160,Journal,1998.0,1998-01-01,2.016092e+08,NaN,NaN,6.0,2.0,16.0,0.0,0.0
41,101311581,10.5070/M3271014515,Journal,1998.0,1998-01-01,1.433928e+08,NaN,NaN,0.0,0.0,0.0,0.0,0.0
101,224584003,10.1007/PL00013476,Journal,1998.0,1998-09-01,9.863128e+07,NaN,NaN,8.0,3.0,2.0,2.0,2.0
119,334129432,NaN,Journal,1991.0,1991-01-01,8.731168e+07,NaN,NaN,8.0,7.0,0.0,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134128996,3091643575,NaN,Journal,1995.0,1995-01-01,6.203030e+07,NaN,NaN,0.0,0.0,0.0,0.0,0.0
134129031,3115556393,NaN,Journal,1998.0,1998-10-01,1.506922e+08,NaN,NaN,0.0,0.0,0.0,0.0,0.0
134129082,3146715096,NaN,Journal,1993.0,1993-01-01,1.162512e+08,NaN,NaN,0.0,0.0,0.0,0.0,0.0
134129083,3148693152,NaN,Journal,1995.0,1995-03-15,1.324228e+07,NaN,NaN,2.0,0.0,0.0,0.0,0.0


In [23]:
%%time
Sel_Year_List = list(df_MAG_J2['Year'].values)
Sel_MagID_List = list(df_MAG_J2['MAGID'].values)
List_Len = len(Sel_Year_List)
from tqdm import tqdm
from collections import defaultdict
Dict_Year_2_MagID = defaultdict(list)
for i in tqdm(range(List_Len)):
    Dict_Year_2_MagID[Sel_Year_List[i]].append(Sel_MagID_List[i])

100%|██████████| 12626981/12626981 [00:05<00:00, 2347439.91it/s]

CPU times: user 5.43 s, sys: 197 ms, total: 5.62 s
Wall time: 6.35 s


In [24]:
print(len(Dict_Year_2_MagID[1998]))
Dict_Year_2_MagID[1998][:10]

1276719


[35428871,
 101311581,
 224584003,
 1538768954,
 1601476401,
 1804039342,
 1809432673,
 1948284696,
 1967056321,
 1967251872]

In [36]:
Cur_Dict = {}
Paper_pair = {}
Tmp_Dict = [{} for i in range(10)]
Tmp_Dict

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]

In [37]:
def cal_J(P_i, P_j, k):
    if P_i in Dict_MagID_2_JID_New and P_j in Dict_MagID_2_JID_New:
        J_i, J_j = Dict_MagID_2_JID_New[P_i], Dict_MagID_2_JID_New[P_j]
        if J_i > J_j:
            J_i, J_j = J_j, J_i
        if (J_i, J_j) in Cur_Dict:
            Cur_Dict[(J_i, J_j)] += 1
        else:
            Cur_Dict[(J_i, J_j)] = 1
        if k in Paper_pair:
            if (J_i, J_j) not in Paper_pair[k]:
                Paper_pair[k].add((J_i, J_j))
        else:
            Paper_pair[k] = set()
            Paper_pair[k].add((J_i, J_j))

In [38]:
def cal_exp_J(P_i, P_j, k):
    if P_i in Dict_MagID_2_JID_New and P_j in Dict_MagID_2_JID_New:
        J_i, J_j = Dict_MagID_2_JID_New[P_i], Dict_MagID_2_JID_New[P_j]
        if J_i > J_j:
            J_i, J_j = J_j, J_i
        if (J_i, J_j) in Tmp_Dict[k]:
            Tmp_Dict[k][(J_i, J_j)] += 1
        else:
            Tmp_Dict[k][(J_i, J_j)] = 1

In [39]:
len(Ori_Dict), len(Dict_Year_2_MagID)

(6236245, 11)

In [42]:
from tqdm import tqdm
import numpy as np
# Z_median_list = defaultdict(list)
# Z_10_percent_list = defaultdict(list)
MAG_Z_median_list = []
MAG_Z_10_pct_list = []
Z_J_Score = []
# epsilon = 1e-4
pas = 0
std_cnt = 0
mean_cnt = 0

Z_for_year = {}

for year, paper_list in Dict_Year_2_MagID.items():
    # k is the paper.
    # v is the reference list.
    Cur_Dict = {}
    Tmp_Dict = [{} for i in range(10)]
    print(year)
    Paper_pair = {}
    Z_for_year = {}
    
    for each_paper in tqdm(paper_list):
        if each_paper not in Ori_Dict:
            continue
            
        k = each_paper
        v = Ori_Dict[each_paper]
        # k is the paper.
        # v is the reference list.
        cur_len = len(v)
        if cur_len > 1000:
            pas += 1
            continue
        for i in range(cur_len):
            for j in range(i):
                cal_J(v[i], v[j], k)

        for p in range(10):
            if k in Ref_List[p]:
                tmp_v = Ref_List[p][k]
                tmp_len = len(tmp_v)
                for i in range(tmp_len):
                    for j in range(i):
                        cal_exp_J(tmp_v[i], tmp_v[j], p)
    
    for k2, v2 in tqdm(Cur_Dict.items()):
        obs = v2
        tmp_Z = []
        for i in range(10):
            if k2 in Tmp_Dict[i]:
                tmp_Z.append(Tmp_Dict[i][k2])
            else:
                tmp_Z.append(0)

        mean = np.mean(tmp_Z)
        std = np.std(tmp_Z)

        if std == 0.0:
            std_cnt += 1
            if mean == 0:
                mean_cnt += 1
            # std += epsilon
        # Noted.
        if std != 0.0:
            Z_for_year[k2] = (obs - mean)/std
            Z_J_Score.append([year, k2[0], k2[1], Z_for_year[k2]])


    for each_paper in tqdm(paper_list):
        k = each_paper
        Z_for_k = []
        if k in Paper_pair:
            for ee in Paper_pair[k]:
                if ee in Z_for_year:
                    Z_for_k.append(Z_for_year[ee])
        # Z_for_k = np.sort(Z_for_k)
        if len(Z_for_k) > 0:
            MAG_Z_median_list.append([k, np.quantile(Z_for_k,0.5), len(Z_for_k)])
            MAG_Z_10_pct_list.append([k, np.quantile(Z_for_k,0.1), len(Z_for_k)])

1997.0


 47%|████▋     | 570505/1221102 [57:03<42:39, 254.21it/s]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 84%|████████▍ | 1025651/1221102 [1:41:52<27:02, 120.46it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 11%|█▏        | 145260/1276719 [12:04<1:43:11, 182.73it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config va

1991.0


100%|██████████| 972467/972467 [01:50<00:00, 8791.18it/s]


1994.0


...

 88%|████████▊ | 838478/954952 [45:27<05:02, 385.40it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 45%|████▌     | 471346/1046074 [28:20<42:23, 225.98it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 40%|████      | 422469/1046074 [00:48<01:07, 9199.21it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variab

1995.0


 11%|█         | 126440/1125219 [08:43<56:22, 295.28it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 1125219/1125219 [02:28<00:00, 7568.34it/s]


1992.0


100%|██████████| 1000433/1000433 [02:00<00:00, 8320.48it/s]


In [43]:
print(pas, std_cnt, mean_cnt)

99 5587802 5587574


In [44]:
print(len(Z_J_Score))

38175430


In [45]:
import pandas as pd
df_MAG_Z_median = pd.DataFrame(MAG_Z_median_list)
df_MAG_Z_median.columns = ['MAGID','Atyp_Z_median','Valid_Pairs']
df_MAG_Z_median

,MAGID,Atyp_Z_median,Valid_Pairs
0,1499627020,2.062288,16
1,1969249394,107.724187,95
2,1969322944,452.636220,7
3,1973645231,130.783213,146
4,1974996827,640.459009,8
...,...,...,...
5818658,1971117140,68.841905,175
5818659,1975985687,118.936101,7
5818660,2039669602,38.124373,23
5818661,2045482264,37.358131,13


In [46]:
df_MAG_Z_median.to_csv('./Data-Metrics/Z_median_1990_2000_MAG_Atyp_Comb.csv', sep='\t', index=False)

In [47]:
import pandas as pd
df_MAG_Z_10_pct = pd.DataFrame(MAG_Z_10_pct_list)
df_MAG_Z_10_pct.columns = ['MAGID','Atyp_Z_10pct','Valid_Pairs']
df_MAG_Z_10_pct

,MAGID,Atyp_Z_10pct,Valid_Pairs
0,1499627020,-1.433254,16
1,1969249394,-5.307769,95
2,1969322944,119.008730,7
3,1973645231,7.540800,146
4,1974996827,254.044920,8
...,...,...,...
5818658,1971117140,-19.962018,175
5818659,1975985687,-1.653492,7
5818660,2039669602,4.449584,23
5818661,2045482264,14.601822,13


In [48]:
df_MAG_Z_10_pct.to_csv('./Data-Metrics/Z_10pct_1990_2000_MAG_Atyp_Comb.csv', sep='\t', index=False)

In [49]:
import pandas as pd
df_Z_J_Score = pd.DataFrame(Z_J_Score)
df_Z_J_Score.columns = ['Year','JID1','JID2','Z_score']
df_Z_J_Score

,Year,JID1,JID2,Z_score
0,1997.0,105380075,207254130,3.000000
1,1997.0,69635808,207254130,0.081923
2,1997.0,69635808,105380075,-2.110579
3,1997.0,157690423,207254130,-0.755929
4,1997.0,105380075,157690423,-3.534133
...,...,...,...,...
38175425,1992.0,57491091,82722058,-6.869196
38175426,1992.0,114911520,127220685,-1.664479
38175427,1992.0,82722058,114911520,-5.752237
38175428,1992.0,158036907,2752809233,-0.081923


In [50]:
df_Z_J_Score.to_csv('./Data-Metrics/Z_Score_Journal_' + str(begin) + '_' + str(end) +'_MAG_Atyp_Comb.csv', 
                       sep='\t', index=False)